In [14]:
import time
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('dark_background')

import h5py
import numpy as np
import tenseal as ts
import torch
import torch.nn as nn
import torch.nn.functional as F
from icecream import ic
ic.configureOutput(includeContext=True)
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, Dataset

print(f'torch version: {torch.__version__}')
print(f'tenseal version: {ts.__version__}')

project_path = Path.cwd().parent
print(f'project_path: {project_path}')

torch version: 1.8.1+cu102
tenseal version: 0.3.5
project_path: /home/dk/Desktop/split-learning-1D-HE


## Dataset

In [15]:
class ECG(Dataset):
    """The class used by the client to load the dataset

    Args:
        Dataset ([type]): [description]
    """
    def __init__(self, train=True):
        if train:
            with h5py.File(project_path/'data/train_ecg.hdf5', 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(project_path/'data/test_ecg.hdf5', 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

In [16]:
batch_size = 16
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [17]:
x_train, y_train = next(iter(train_loader))
print(x_train.size())
print(y_train.size())
total_batch = len(train_loader)
print(total_batch)

torch.Size([16, 1, 128])
torch.Size([16])
828


## Training loop

In [18]:
epoch = 10
lr = 0.001
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f'device: {torch.cuda.get_device_name(0)}')

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def train(model, save_weight_path: str):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    train_losses = list()
    train_accs = list()
    train_times = list()

    test_losses = list()
    test_accs = list()
    test_times = list()

    best_test_acc = 0  # best test accuracy 

    for e in range(epoch):
        train_start = time.time()
        print("Epoch {} - ".format(e+1), end='')

        # train
        train_loss = 0.0
        correct, total = 0, 0
        for i, batch in enumerate(train_loader):
            x, label = batch  # get feature and label from a batch
            x, label = x.to(device), label.to(device)  # send to device
            optimizer.zero_grad()  # init all grads to zero
            output = model(x)  # forward propagation
            loss = criterion(output, label)  # calculate loss
            loss.backward()  # backward propagation
            optimizer.step()  # weight update

            train_loss += loss.item()
            correct += torch.sum(output.argmax(dim=1) == label).item()
            total += len(label)

        train_losses.append(train_loss / len(train_loader))
        train_accs.append(correct / total)
        train_end = time.time()
        train_times.append(train_end-train_start)
        print("loss: {:.4f}, acc: {:.2f}%".format(train_losses[-1], train_accs[-1]*100), end=' / ')
    
        # test
        test_start = time.time()
        with torch.no_grad():
            test_loss = 0.0
            correct, total = 0, 0
            for _, batch in enumerate(test_loader):
                x, label = batch
                x, label = x.to(device), label.to(device)
                output = model(x)
                loss = criterion(output, label)
                
                test_loss += loss.item()
                correct += torch.sum(output.argmax(dim=1) == label).item()
                total += len(label)
            test_losses.append(test_loss / len(test_loader))
            test_acc = correct / total
            test_accs.append(test_acc)
        test_end = time.time()
        test_times.append(test_end-test_start)
        print("test_loss: {:.4f}, test_acc: {:.2f}%".format(test_losses[-1], test_accs[-1]*100))
        if test_accs[-1] > best_test_acc:
            best_test_acc = test_acc
            torch.save(ecgnet.state_dict(), save_weight_path)  # save trained weights
            print(f"found new best test accuracy: {test_acc*100:.2f}%, save model")

    return train_losses, train_accs, train_times, test_losses, test_accs, test_times

device: NVIDIA GeForce GTX 1070 Ti


## Models

### Model 1: 512 (activation maps length)

In [19]:
class ECGConv1D512(nn.Module):
    def __init__(self):
        super(ECGConv1D512, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 16 = 512

        self.linear = nn.Linear(32 * 16, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 16)
        x = self.linear(x)
        x = self.softmax(x)
        return x    

ecgnet = ECGConv1D512()
ecgnet = ecgnet.to(device)
train_losses, train_accs, train_times, \
    test_losses, test_accs, test_times = train(ecgnet, 'weights/trained_weight_512_1_bz1.pth')    

Epoch 1 - loss: 1.3313, acc: 58.86% / test_loss: 1.2179, test_acc: 70.62%
found new best test accuracy: 70.62%, save model
Epoch 2 - loss: 1.1869, acc: 73.36% / test_loss: 1.1478, test_acc: 76.69%
found new best test accuracy: 76.69%, save model
Epoch 3 - loss: 1.1487, acc: 76.43% / test_loss: 1.1213, test_acc: 78.88%
found new best test accuracy: 78.88%, save model
Epoch 4 - loss: 1.1166, acc: 79.49% / test_loss: 1.0987, test_acc: 80.87%
found new best test accuracy: 80.87%, save model
Epoch 5 - loss: 1.1052, acc: 80.33% / test_loss: 1.0936, test_acc: 81.33%
found new best test accuracy: 81.33%, save model
Epoch 6 - loss: 1.0996, acc: 80.75% / test_loss: 1.0910, test_acc: 81.51%
found new best test accuracy: 81.51%, save model
Epoch 7 - loss: 1.0961, acc: 80.94% / test_loss: 1.0883, test_acc: 81.65%
found new best test accuracy: 81.65%, save model
Epoch 8 - loss: 1.0731, acc: 83.55% / test_loss: 1.0472, test_acc: 86.01%
found new best test accuracy: 86.01%, save model
Epoch 9 - loss: 

### Model 2: 128

In [12]:
class ECGConv1D128(nn.Module):
    def __init__(self):
        super(ECGConv1D128, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3,
                               stride=1)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=4, 
                               kernel_size=5, 
                               padding=2,
                               stride=1)  # 64 x 4
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 4 = 128
        
        self.linear = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 128)
        x = self.linear(x)
        x = self.softmax(x)
        return x

ecgnet = ECGConv1D128()
train_losses, train_accs, train_times, \
    test_losses, test_accs, test_times = train(ecgnet.to(device), 
                        'weights/trained_weight_128_1.pth')

Epoch 1 - loss: 1.3177, acc: 60.60% / test_loss: 1.1814, test_acc: 74.59%
found new best test accuracy: 74.59%, save model
Epoch 2 - loss: 1.1689, acc: 74.51% / test_loss: 1.1482, test_acc: 76.35%
found new best test accuracy: 76.35%, save model
Epoch 3 - loss: 1.1500, acc: 75.91% / test_loss: 1.1298, test_acc: 77.67%
found new best test accuracy: 77.67%, save model
Epoch 4 - loss: 1.1302, acc: 77.60% / test_loss: 1.1152, test_acc: 78.99%
found new best test accuracy: 78.99%, save model
Epoch 5 - loss: 1.1185, acc: 78.72% / test_loss: 1.1052, test_acc: 80.00%
found new best test accuracy: 80.00%, save model
Epoch 6 - loss: 1.1127, acc: 79.28% / test_loss: 1.1015, test_acc: 80.23%
found new best test accuracy: 80.23%, save model
Epoch 7 - loss: 1.1053, acc: 80.08% / test_loss: 1.0932, test_acc: 81.11%
found new best test accuracy: 81.11%, save model
Epoch 8 - loss: 1.1009, acc: 80.39% / test_loss: 1.0888, test_acc: 81.43%
found new best test accuracy: 81.43%, save model
Epoch 9 - loss: 

## Model 3: 64

In [13]:
class ECGConv1D64(nn.Module):
    def __init__(self):
        super(ECGConv1D64, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3,
                               stride=1)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=4, 
                               kernel_size=5, 
                               padding=2,
                               stride=1)  # 64 x 4
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 4 = 128
        
        self.linear = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 128)
        x = self.linear(x)
        x = self.softmax(x)
        return x

ecgnet = ECGConv1D64()
train_losses, train_accs, train_times, \
    test_losses, test_accs, test_times = train(ecgnet.to(device), 
                            'weights/trained_weight_64_1.pth')

Epoch 1 - loss: 1.3498, acc: 57.58% / test_loss: 1.2441, test_acc: 67.63%
found new best test accuracy: 67.63%, save model
Epoch 2 - loss: 1.2282, acc: 68.46% / test_loss: 1.1948, test_acc: 71.20%
found new best test accuracy: 71.20%, save model
Epoch 3 - loss: 1.1818, acc: 73.70% / test_loss: 1.1568, test_acc: 75.57%
found new best test accuracy: 75.57%, save model
Epoch 4 - loss: 1.1608, acc: 75.22% / test_loss: 1.1531, test_acc: 75.82%
found new best test accuracy: 75.82%, save model
Epoch 5 - loss: 1.1497, acc: 76.01% / test_loss: 1.1481, test_acc: 76.03%
found new best test accuracy: 76.03%, save model
Epoch 6 - loss: 1.1406, acc: 76.76% / test_loss: 1.1328, test_acc: 77.30%
found new best test accuracy: 77.30%, save model
Epoch 7 - loss: 1.1367, acc: 76.96% / test_loss: 1.1275, test_acc: 77.83%
found new best test accuracy: 77.83%, save model
Epoch 8 - loss: 1.1328, acc: 77.33% / test_loss: 1.1249, test_acc: 78.11%
found new best test accuracy: 78.11%, save model
Epoch 9 - loss: 